<div style="overflow: hidden;">
    <img src="images/DREGS_logo_v2.png" width="300" style="float: left; margin-right: 10px;">
</div>

# Getting started: Part X - Using dataregistry with GCRCatalogs

Here we show how to access catalogs belonging to the `GCRCatalogs` package via the information stored in the dataregistry.

### What we cover in this tutorial

In this tutorial we will learn how to:

1) Find and read the catalogs using the standard GCRCatalogs interface
2) Query catalog metadata directly using the data registry, then use that metadata to find and read catalogs

### Before we begin

Currently (November, 2024) the required versions of gcr-catalogs and dataregistry are only available in the `desc-python-bleed` kernel. Make sure you have selected that kernel while running this tutorial.

If you haven't done so already, check out the [getting setup](https://lsstdesc.org/dataregistry/tutorial_setup.html) page from the documentation if you want to run this tutorial interactively.

## 1) Using the usual GCRCatalogs interface

Note that, using this method, we will not be calling any data registry services directly, but the data registry database still must be accessible. That means you must have gone through at least part of the tutorial setup referred to above, in particular the steps for creating a couple small files needed for authentication. See details [here](http://lsstdesc.org/dataregistry/installation.html#one-time-setup).

### Configuring GCRCatalogs 

A quick way to check everything is set up correctly is to run the first cell below, which should load the GCRCatalogs package, and print the package version. It should be at least 1.9.0.

In [ ]:
import GCRCatalogs
print(f"Working with GCRCatalogs version: {GCRCatalogs.__version__}")

We need to tell `GCRCatalogs` whether to use the old-style metadata access method (reading config files) or to fetch metadata from the data registry.  There are two ways to do this:

1. Before running, set the environment variable `GCR_CONFIG_SOURCE`to one of the two allowed values: "files" or "dataregistry"
2. Invoke the GCRCatalogs routine `ConfigSource.set_config_source`

Here we use the second method.


In [ ]:
# Tell GCRCatalogs to use the data registry
GCRCatalogs.ConfigSource.set_config_source(dr=True)

Now we can use any of the standard GCRCatalogs query routines.

In [ ]:
# Find catalogs whose name starts with "cosmo"
cosmos = GCRCatalogs.get_available_catalog_names(name_startswith="cosmo")
cosmos

In [ ]:
# Load a catalog; find out something about it
cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_small")
cat.native_filter_quantities

## 2) Using the data registry directly

We learned how to connect to the DESC data registry in other tutorials using the `DataRegistry` class. Let's connect again using the defaults _except_ for the schema.  Since the catalogs maintained by GCRCatalogs are stored in the DESC production shared area, their database entries are in the production schema, not in the (default) working schema.



In [ ]:
from dataregistry import DataRegistry
from dataregistry.schema import DEFAULT_SCHEMA_PRODUCTION

# Establish connection to the production schema
datareg = DataRegistry(schema=DEFAULT_SCHEMA_PRODUCTION)

### Dataset properties

Recall that a `DataRegistry` instance has a member `Query` which provides all the query services, the principal one being the ability to ask for values of attributes of datasets, subject to one or more filters.  If you haven't already, we recommend you take a look at the tutorial "Getting started: Part 3 - Simple queries" before proceeding further.

You can find out what the dataset properties ("columns" in database parlance) are with another of the `Query` services: 

In [ ]:
all_columns = datareg.Query.get_all_columns()
print(all_columns)

That is a list of __all__ columns from __all__ tables, maybe more than we bargained for. Let's restrict it to columns in the `dataset` table.

In [ ]:
dataset_columns = [col for col in all_columns if col.startswith('dataset.')]
print(dataset_columns)

Among the more interesting for our purposes are `name`, `relative_path`, `access_api`, `access_api_configuration` and `location_type`. In the case of catalogs registered with GCRCatalogs, `name` in the data registry is the same name GCRCatalogs uses to refer to it: the basename of the corresponding config file, not including the suffix `.yaml`.  But keep in mind that, unlike GCRCatalog, the dataregistry always respects case in names

Let's look at those properties for the dataset `cosmoDC2_v1.1.4`.

In [ ]:
# Define a filter on the "name" property and make query
from dataregistry.query import Filter
catname = 'cosmoDC2_v1.1.4'
filters = [Filter('dataset.name', '==', catname)]
property_names = ['dataset.name', 'dataset.relative_path', 'dataset.access_api', 
                  'dataset.access_api_configuration', 'dataset.location_type']
result = datareg.Query.find_datasets(property_names=property_names,
                                     filters=filters)
# By default the return type is a dict
for k, v in result.items():
    print(f'Key {k} has value \n{v[0]}\n')


At NERSC (currently the only place this code can be run) the value for `relative_path` is relative to the DESC NERSC production shared area, `/global/cfs/cdirs/lsst/shared`, just like the path names used in GCRCatalogs. 

In [ ]:
import os
abs_path = os.path.join('/global/cfs/cdirs/lsst/shared',
        result['dataset.relative_path'][0])
abs_path

The value "GCRCatalogs" for the property `dataset.access_api` is a clue that this
dataset may be read and interpreted using GCRCatalogs.

The value for `dataset.access_api_configuration` should look familiar.  It's just the contents of this catalogs's config file. And the value for the location type, "dataregistry", just tells us this is a normal catalog whose data files are kept in the area managed by the data registry.

Let's try this for another catalog. We'll just change the name and make the same query.

In [ ]:
catname = 'cosmoDC2_v1.1.4_small'
filters = [Filter('dataset.name', '==', catname)]
property_names = ['dataset.name', 'dataset.relative_path', 'dataset.access_api', 
                  'dataset.access_api_configuration', 'dataset.location_type']
result = datareg.Query.find_datasets(property_names=property_names,
                                     filters=filters)
# By default the return type is a dict
for k, v in result.items():
    print(f'Key {k} has value \n{v[0]}\n')

It all looks pretty much as you would expect, except what happened to the value of `dataset.relative_path`?   That doesn't look like a path. You can see the reason in the catalog's configuration:  it's based on another catalog. Or you can see it in the value for `dataset.location_type`. "meta_only" means that the data registry is only storing metadata for the catalog; it is not keeping track of the (indirectly) associated files.  The same thing would happen for a composite catalog: the data registry just stores the catalog's configuration. It doesn't know how to parse it as GCRCatalogs would.